In [ ]:
lakehouseName = "lakehouse name"

StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [ ]:
# Helper functions

import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType, DecimalType, BinaryType
spark.conf.set('spark.sql.caseSensitive', True)

def remap(df, columnMap):
    dfRenamed = df.rename(columns=columnMap)
    dfRenamed = dfRenamed[columnMap.values()]
    df = spark.createDataFrame(dfRenamed) 
    return df

def renameDf(df, renameList):
    for col in renameList.keys():
        df = df.withColumnRenamed(col, renameList[col])
    return df

def retype(df, retypes):
    for col in retypes.keys():
        df = df.withColumn(col, df[col].cast(retypes[col]))
    return df

def save(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .mode("overwrite") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

def saveWithSchemaOverwrite(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

def saveWithSchemaMerge(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .option("mergeSchema", "true") \
        .mode("overwrite") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
import pyspark.sql.functions as F

# Currency, Store, Location, Country and Order tables
dfCurrency = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Currency.csv")

dfLocation = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Location.csv")
colRetype={"LocationId":"string", "CountryId":"string"}
dfLocation = retype(dfLocation, colRetype);

dfCountry = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Country.csv")
colRetype={"CountryId":"string"}
dfCountry = retype(dfCountry, colRetype);

dfOrder = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Order.csv")
colRetype={"OrderId":"string", "OrderTotalAmount":"decimal(18,2)", "OrderActualDeliveryTimestamp":"timestamp", "StoreId":"string", "OrderRecievedTimeStamp":"timestamp"}
dfOrder = retype(dfOrder, colRetype);

dfStore = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Store.csv")
colRetype={"StoreId":"string", "LocationId":"string", "StoreOpenDate":"date", "GrossFloorArea":"int", "MerchantId":"string"}
dfStore = retype(dfStore, colRetype);

# Persist normalized tables
saveWithSchemaMerge(dfCurrency,  "Currency")
saveWithSchemaMerge(dfLocation,  "Location")
saveWithSchemaMerge(dfCountry,  "Country")
saveWithSchemaMerge(dfOrder,  "Order")
saveWithSchemaMerge(dfStore,  "Store")

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
print(dfCurrency.count())
print(dfLocation.count())
print(dfCountry.count())
print(dfOrder.count())
print(dfStore.count())

StatementMeta(, , , Waiting, , Waiting)